<a href="https://colab.research.google.com/github/haniklakhe18/Temperature_analysis/blob/main/2_Ensemble_database_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/Indices/Ensemble'

/content/drive/.shortcut-targets-by-id/1BBkjA5D9rIVQl4WqjLSgE0LYjPK4dPfq/Indices/Ensemble


In [ ]:
!pip install geopandas
!pip install rasterio
!pip install pyidw
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
import numpy as np
import fiona
from sklearn.metrics import mean_squared_error
import rasterio
import rasterio.plot
import rasterio.mask
import glob
from tqdm import tqdm
import os
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
CM = ['ACCESS-CM2','ACCESS-ESM1-5','BCC-CSM2-MR','CanESM5','EC-Earth3','EC-Earth3-Veg','INM-CM4-8',
      'INM-CM5-0','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NorESM2-LM','NorESM2-MM']
#CM = ['ACCESS-CM2']

scenario = ['historical','ssp126', 'ssp245', 'ssp370', 'ssp585']

In [ ]:
df = pd.read_csv('Location.csv')
df['Station'] = df['Station'].astype(str)
print(df.dtypes)

Station     object
Lon        float64
Lat        float64
dtype: object


In [ ]:
for i in tqdm(range(len(CM))):
    historical_path = glob.glob('Computed_Indices/' + CM[i] + '/historical/' + '*.csv')
    historical_files = sorted(historical_path, key=lambda path: os.path.basename(path))
    for j in range(1, len(scenario)):
        file_path = str(CM[i]) + '/' + str(scenario[j]) + '/'
        scenario_files = glob.glob('Computed_Indices' + '/' + file_path + '*.csv')
        scenario_files = sorted(scenario_files, key=lambda path: os.path.basename(path))
        for historical_file, scenario_file in zip(historical_files, scenario_files):
            historical_df = pd.read_csv(historical_file)
            historical_df_transpose = historical_df.transpose()
            col_names = historical_df_transpose.iloc[0].astype(int)
            historical_df_transpose = historical_df_transpose[1:]
            historical_df_transpose.columns = col_names
            historical_df_transpose = historical_df_transpose.reset_index()
            historical_df_transpose.rename(columns={'index': 'Station'}, inplace=True)
            scenario_df = pd.read_csv(scenario_file)
            scenario_df_transpose = scenario_df.transpose()
            col_names = scenario_df_transpose.iloc[0].astype(int)
            scenario_df_transpose = scenario_df_transpose[1:]
            scenario_df_transpose.columns = col_names
            scenario_df_transpose = scenario_df_transpose.reset_index()
            scenario_df_transpose.rename(columns={'index': 'Station'}, inplace=True)
            merged_df = pd.merge(historical_df_transpose, scenario_df_transpose, on = 'Station')
            merged_df['1985-2014'] = np.ceil(merged_df.iloc[:, 35:65].mean(axis=1))
            merged_df['2026-2055'] = np.ceil(merged_df.iloc[:, 76:106].mean(axis=1))
            merged_df['2071-2100'] = np.ceil(merged_df.iloc[:, 121:151].mean(axis=1))
            merged_df['Deviation_NF'] =  merged_df['2026-2055'] - merged_df['1985-2014']
            merged_df['Deviation_FF'] =  merged_df['2071-2100'] - merged_df['1985-2014']
            merged_df.to_csv('Database/' + scenario_file[17:], index = False)


100%|██████████| 13/13 [32:16<00:00, 148.92s/it]


In [ ]:
CM_dict = {}
for c in tqdm(range(0,len(CM))):
    CM_dict[CM[c]] = {}
    for s in range(1,len(scenario)):
        ssp_dict = {}
        CM_dict[CM[c]][scenario[s]] = ssp_dict
        file_path = str(CM[c]) + '/' + str(scenario[s]) + '/'
        indices_files = glob.glob('Database' + '/' + file_path + '*.csv')
        for i in range(0, len(indices_files)):
            indices = pd.read_csv(indices_files[i])
            # Extract the filename without the extension and use it as the key
            base_name = os.path.basename(indices_files[i])
            indice_name = os.path.splitext(base_name)[0]
            #print(indice_name)
            ssp_dict[indice_name] = indices

# Now you can access the data using the nested dictionary structure
# For example: CM_dict['ACCESS-CM2']['ssp126']['DTR']

100%|██████████| 13/13 [00:35<00:00,  2.71s/it]


In [ ]:
indices = list(CM_dict['ACCESS-CM2']['ssp126'].keys())
#indices = ['SU']
for i in tqdm(range(0, len(indices))):
  for s in range(1, len(scenario)):
    dfs = []
    for c in range(0, len(CM)):
      data = CM_dict[CM[c]][scenario[s]][indices[i]]
      dfs.append(data)

      #Stores all the dataframe in dfs as numpy array in the form of list
      arrays = [df.to_numpy() for df in dfs]

      # Now a columnwise database is made all the first column is stored as new array /Stack the arrays along a new axis (axis=0)
      stacked_arrays = np.stack(arrays, axis=0) #look more into np.stack

      # Calculate the median along the first axis (axis=0) to get median for each cell
      median_array = np.median(stacked_arrays, axis=0)

      # Convert the median array back to a DataFrame
      median_df = pd.DataFrame(median_array, columns=dfs[0].columns)
      median_df['1985-2014'] = np.ceil(median_df.iloc[:, 35:65].mean(axis=1))
      median_df['2026-2055'] = np.ceil(median_df.iloc[:, 76:106].mean(axis=1))
      median_df['2071-2100'] = np.ceil(median_df.iloc[:, 121:151].mean(axis=1))
      median_df['Deviation_NF'] = median_df['2026-2055'] - median_df['1985-2014']
      median_df['Deviation_FF'] = median_df['2071-2100'] - median_df['1985-2014']
      median_df.to_csv('Database/' + 'Ensemble_median/' + scenario[s] + '/' + indices[i] + '.csv', index = False)

100%|██████████| 30/30 [02:32<00:00,  5.07s/it]
